In [ ]:
from time import sleep
import pandas as pd
import requests
import json
from PIL import Image
from base64 import b64encode,b64decode
from io import BytesIO
from bs4 import BeautifulSoup
import threading
from requests.adapters import HTTPAdapter

df = pd.read_csv('account.csv')

In [ ]:
def vote(account,password):
    read_time = 20
    delay = 5
    img_flag = False
    token_flag = False
    sign_flag = False
    x_csrf_flag = False
    vote_flag = False
    s = requests.session()
    s.mount('http://', HTTPAdapter(max_retries=30))
    s.mount('https://', HTTPAdapter(max_retries=30))
    
    while token_flag == False:
        try:
            r =s.get("https://youthdream.phdf.org.tw/member/login",timeout=read_time)
            if r.status_code == 200:
                soup = BeautifulSoup(r.text, 'html.parser')
                _token = soup.select_one('input[name = "_token"]')['value']
                token_flag = True
            sleep(delay)
        except:
            None
    ## get cpatcha_url
    _ = soup.select_one('a[id = "captcha"]').select_one('img')
    captcha_url = _["src"]
    
    while sign_flag == False:
        img_flag = False
        ## get captcha_img   
        while img_flag == False:
            try:
                cap_img =s.get(captcha_url,timeout=read_time)
                if cap_img.status_code == 200:
                    image = Image.open(BytesIO(cap_img.content))
                    #image.show()
                    image = image.convert('RGB')
                    image = image.resize((160,60))
                    img_flag = True
                sleep(delay)
            except:
                None
        
        ## break captcha
        ENCODING = 'utf-8'
        output_buffer = BytesIO()
        image.save(output_buffer, format='JPEG')
        byte_data = output_buffer.getvalue()
        base64_bytes = b64encode(byte_data)
        base64_string = base64_bytes.decode(ENCODING)
        data = {"Captcha":base64_string}  # img是ndarray，無法直接用base64編碼，否則會報錯
        data = json.dumps(data)
        r = s.post(" https://captchabreak-261615.appspot.com/Captcha", data=data,timeout=read_time)
        c = json.loads(r.text)["Captcha"]

        
        ## login
        data = {
            "_token":_token,
            "email": account,
            "password": password,
            "captcha": c
        }
        try:
            r = s.post("https://youthdream.phdf.org.tw/member/login",data = data,timeout=read_time)
        except:
            None
            
        if r.status_code == 200:
            print(account," Sign in sucessfully!")
            sign_flag = True
        sleep(delay)
        
    ## get x-csrf-token
    while x_csrf_flag == False:
        try:
            r = s.get("https://youthdream.phdf.org.tw/member/profile",timeout=read_time)
            if r.status_code == 200:
                x_csrf_flag = True
                soup = BeautifulSoup(r.text, 'html.parser')
                csrf_token = soup.select_one('meta[name="csrf-token"]')['content']
            sleep(delay)
        except:
            None
    ## vote
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; WOW64; rv:53.0) Gecko/20100101 Firefox/53.0",
        "X-CSRF-TOKEN":csrf_token,
        "Accept": "*/*",
        "Connection": "keep-alive",
        "Content-Length": "0",
        "Host": "youthdream.phdf.org.tw",
        "Origin":"https://youthdream.phdf.org.tw",
        "Sec-Fetch-Mode": "cors",
        "X-Requested-With": "XMLHttpRequest"
    }
    while vote_flag == False:
        try:
            r = s.post("https://youthdream.phdf.org.tw/project/83",headers = headers,timeout=read_time)
            r2 = s.post("https://youthdream.phdf.org.tw/project/82",headers = headers,timeout=read_time)
            if r.status_code == 200 and r2.status_code == 200:
                vote_flag =True
                print(account," message = ",json.loads(r2.text))
            sleep(delay)
        except:
            None
    ## end
    s.close()